In [2]:
from __future__ import print_function

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn

tf.set_random_seed(777)

In [4]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
seq_length = 10  # Any arbitrary number
learning_rate = 0.1

In [5]:
dataX = []
dataY = []

for i in range(0, len(sentence) - seq_length):
    x_str = sentence[i:i + seq_length]
    y_str = sentence[i+1:i + seq_length + 1]
    print(i, x_str, '->', y_str)
    
    x = [char_dic[c] for c in x_str]
    y = [char_dic[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [6]:
batch_size = len(dataX)

X = tf.placeholder(tf.int32, [None, seq_length])
Y = tf.placeholder(tf.int32, [None, seq_length])

# One-hot
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)

Tensor("one_hot:0", shape=(?, 10, 25), dtype=float32)


In [7]:
def lstm_cell():
    cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    return cell

multi_cells = rnn.MultiRNNCell([lstm_cell() for _ in range(2)], state_is_tuple=True)

outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)

In [8]:
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

outputs = tf.reshape(outputs, [batch_size, seq_length, num_classes])

In [10]:
weights = tf.ones([batch_size, seq_length])

seq_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(seq_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l, results = sess.run([train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), 1)

0 0 yyykkklll' 1
0 1 dlluulll.. 1
0 2 wiuggg...o 1
0 3 kuaao.oooo 1
0 4 gggi..ooog 1
0 5 iiiioooigg 1
0 6 wi.... ggg 1
0 7 p.... ugug 1
0 8 hbbueegggg 1
0 9 ookkkkgggk 1
0 10  nnggggggl 1
0 11 wigggggggl 1
0 12  gggggg'l' 1
0 13 gggga''''' 1
0 14 wuaaiii''' 1
0 15 uaaiii'''p 1
0 16 iiiiiii'pp 1
0 17 y''k''''dd 1
0 18 kkl'''ppdd 1
0 19 al'''pp'dy 1
0 20 w'''ppdddy 1
0 21 hhrpppddyy 1
0 22 wlppddyyyy 1
0 23 ppppdyyyyu 1
0 24 pppyyyyyoo 1
0 25 yyyyyygugg 1
0 26 nyyyyggggg 1
0 27 yyyguuuuuu 1
0 28 wlluooukul 1
0 29 auuoookkkl 1
0 30 gboookkkll 1
0 31 oookkkklll 1
0 32 o  klllllc 1
0 33  niniciiii 1
0 34 wlyyyyyyyy 1
0 35 ayyyyiiiii 1
0 36 yyyyiiiiic 1
0 37 iciiiiiiic 1
0 38 gggggyiggg 1
0 39 wiiiiiicgg 1
0 40 iiiiiiccci 1
0 41 nnnnnggggg 1
0 42 wyiggggggg 1
0 43 nnnnccwggg 1
0 44 wgggkwgggg 1
0 45 gg'ggggggg 1
0 46 n'iiiggggg 1
0 47 kkkiggggkk 1
0 48 wwigggggkk 1
0 49 wigggggkkw 1
0 50  g,kkkkkkk 1
0 51 g,wwckcwic 1
0 52 www  iiick 1
0 53 w    iikkk 1
0 54     iickgg 1
0 55 p  iicgggg 1
0 

3 0 t  eeeeeee 1
3 1 t  eeeeeee 1
3 2 t  eeeeeee 1
3 3 t  eeeeeee 1
3 4 t  eeeeeee 1
3 5 t  eeeeeee 1
3 6 t  eeeeeee 1
3 7 t  eeeeeee 1
3 8 t  eeeeeee 1
3 9 t  eeeeeee 1
3 10 t  eeeeeee 1
3 11 t  eeeeeee 1
3 12 t  eeeeeee 1
3 13 t  eeeeeee 1
3 14 tt eeeeeee 1
3 15 t  eeeeeee 1
3 16 t  eeeeeee 1
3 17 t  eeeeeee 1
3 18 t  eeeeeee 1
3 19 tt eeeeeee 1
3 20 tt eeeeeee 1
3 21 tt eeeeeee 1
3 22 t  eeeeeee 1
3 23 t  eeeeeee 1
3 24 t  eeeeeee 1
3 25 t  eeeeeee 1
3 26 tt eeeeeee 1
3 27 tt eeeeeee 1
3 28 tt eeeeeee 1
3 29 t  eeeeeee 1
3 30 t  eeeeeee 1
3 31 t  eeeeeee 1
3 32 t  eeeeeee 1
3 33 t  eeeeeee 1
3 34 tt eeeeeee 1
3 35 t  eeeeeee 1
3 36 t  eeeeeee 1
3 37 t  eeeeeee 1
3 38 t  eeeeeee 1
3 39 t  eeeeeee 1
3 40 t  eeeeeee 1
3 41 tt eeeeeee 1
3 42 t  eeeeeee 1
3 43 t  eeeeeee 1
3 44 t  eeeeeee 1
3 45 t  eeeeeee 1
3 46 t  eeeeeee 1
3 47 t  eeeeeee 1
3 48 t  eeeeeee 1
3 49 t  eeeeeee 1
3 50 t  eeeeeee 1
3 51 t  eeeeeee 1
3 52 t   eeeeee 1
3 53 t  eeeeeee 1
3 54 t  eeeeeee 1
3 55 t  eeeeeee 1
3 

8 76 t          1
8 77 t          1
8 78 t          1
8 79 t          1
8 80 t          1
8 81 t          1
8 82 t          1
8 83 t          1
8 84 t          1
8 85 t          1
8 86 t          1
8 87 t          1
8 88 t          1
8 89 t          1
8 90 t          1
8 91 t          1
8 92 t          1
8 93 t          1
8 94 t          1
8 95 t          1
8 96 t          1
8 97 t          1
8 98 t          1
8 99 t          1
8 100 t          1
8 101 t          1
8 102 t          1
8 103 t          1
8 104 t          1
8 105 t          1
8 106 t          1
8 107 t          1
8 108 t          1
8 109 t          1
8 110 t          1
8 111 t          1
8 112 t          1
8 113 t          1
8 114 t          1
8 115 t          1
8 116 t          1
8 117 t          1
8 118 t          1
8 119 t          1
8 120 t          1
8 121 t          1
8 122 t          1
8 123 t          1
8 124 t          1
8 125 t          1
8 126 t          1
8 127 t          1
8 128 t          1
8 129 t          

13 101            1
13 102            1
13 103            1
13 104            1
13 105          t 1
13 106         t  1
13 107            1
13 108            1
13 109            1
13 110            1
13 111            1
13 112            1
13 113            1
13 114            1
13 115            1
13 116            1
13 117            1
13 118            1
13 119            1
13 120            1
13 121            1
13 122            1
13 123            1
13 124            1
13 125            1
13 126            1
13 127            1
13 128            1
13 129            1
13 130            1
13 131            1
13 132            1
13 133            1
13 134            1
13 135            1
13 136            1
13 137            1
13 138            1
13 139            1
13 140            1
13 141            1
13 142            1
13 143            1
13 144            1
13 145            1
13 146            1
13 147            1
13 148            1
13 149            1
13 150            1


16 91 t          1
16 92            1
16 93          t 1
16 94         tt 1
16 95        ttt 1
16 96 t     ttt  1
16 97      ttt t 1
16 98    tttt tt 1
16 99    ttt tt  1
16 100   tt  tt   1
16 101  tt  tt tt 1
16 102 ttt tt tt  1
16 103    tt tt   1
16 104   tt tt  t 1
16 105   tttt  tt 1
16 106 t ttt ttt  1
16 107    t  tt   1
16 108   tt tt    1
16 109     tt   t 1
16 110    tt   t  1
16 111   tt   t   1
16 112 ttt   t    1
16 113 tt   t     1
16 114     t    t 1
16 115    t    t  1
16 116 t t    t   1
16 117       t    1
16 118      t     1
16 119     t      1
16 120    tt      1
16 121   tt       1
16 122  tt        1
16 123 t   t      1
16 124            1
16 125            1
16 126            1
16 127            1
16 128            1
16 129 t        t 1
16 130         t  1
16 131        t   1
16 132       t  t 1
16 133     tt  t  1
16 134 t  tt  t   1
16 135    t  t    1
16 136   t  t     1
16 137 t   t   tt 1
16 138         t  1
16 139   tt   t   1
16 140   t   t    1
16 141   

...

484 12 h cuild a  1
484 13  cuild a s 1
484 14 tutld a sh 1
484 15 utld a shi 1
484 16 pld a ship 1
484 17 ld a ship, 1
484 18 e a ship,  1
484 19  anship, d 1
484 20 tnship, do 1
484 21 nship, don 1
484 22 thip, don' 1
484 23 sip, don't 1
484 24 ep, don't  1
484 25 l, don't d 1
484 26 e don't dr 1
484 27  bon't dru 1
484 28 ton't arum 1
484 29  n't arum  1
484 30  't arum u 1
484 31 dt arum up 1
484 32 t arum up  1
484 33 harum up p 1
484 34 toum up pe 1
484 35  um up peo 1
484 36  m up peop 1
484 37 p up peopl 1
484 38  tp people 1
484 39 tp people  1
484 40 p people t 1
484 41 epeople to 1
484 42 teople tog 1
484 43 eople toge 1
484 44 rple toget 1
484 45  le togeth 1
484 46 ee togethe 1
484 47 ectogether 1
484 48 reogether  1
484 49 to ether t 1
484 50 h ether to 1
484 51  ether to  1
484 52 ether to c 1
484 53 rher to co 1
484 54 hem to col 1
484 55 er to coll 1
484 56 r to colle 1
484 57  to collec 1
484 58 to collect 1
484 59 h collect  1
484 60  collect w 1
484 61 tollect wo 1


487 127 t them to  1
487 128 ethem to l 1
487 129 toem ta lo 1
487 130  er ta lon 1
487 131 em ta long 1
487 132   ta long  1
487 133  ta long f 1
487 134 to cong fo 1
487 135   cong for 1
487 136 nbong for  1
487 137 tong for t 1
487 138 eng for th 1
487 139 n' for the 1
487 140 d for the  1
487 141 nfor the e 1
487 142 tor the en 1
487 143  r the end 1
487 144 n the endl 1
487 145  the endle 1
487 146 toemendles 1
487 147  erendless 1
487 148 emendless  1
487 149  tndless i 1
487 150 tndless im 1
487 151  sless imm 1
487 152 d ess imme 1
487 153  ess immen 1
487 154 e s immens 1
487 155  s immensi 1
487 156   immensit 1
487 157  immensity 1
487 158 tmmensity  1
487 159 pmensity o 1
487 160  ensity of 1
487 161  nsity of  1
487 162  sity of t 1
487 163 dity of th 1
487 164  ty of the 1
487 165 py of the  1
487 166   of the s 1
487 167  of the se 1
487 168 tf the sea 1
487 169 n the sea. 1
488 0 l you want 1
488 1  you want  1
488 2 tou want t 1
488 3  u want to 1
488 4   want to  1
48

490 116 tather tea 1
490 117  ther teac 1
490 118 nher teach 1
490 119  er teach  1
490 120 em teach t 1
490 121   teach th 1
490 122  thach the 1
490 123 toach them 1
490 124  ach them  1
490 125  ch them t 1
490 126 nh them to 1
490 127 t them to  1
490 128 ethem to l 1
490 129 toem ta lo 1
490 130  er ta lon 1
490 131 em ta long 1
490 132   ta long  1
490 133  ta long f 1
490 134 to cong fo 1
490 135   cong for 1
490 136  long for  1
490 137 tong for t 1
490 138 eng for th 1
490 139  ' for the 1
490 140 d for the  1
490 141 nfor the e 1
490 142 tor the en 1
490 143  r the end 1
490 144   the endl 1
490 145  the endle 1
490 146 toemendles 1
490 147  erendless 1
490 148 emendless  1
490 149  tndless i 1
490 150 tndless im 1
490 151  dless imm 1
490 152 d ess imme 1
490 153  ess immen 1
490 154 e s immens 1
490 155  s immensi 1
490 156   immensit 1
490 157  immensity 1
490 158 tmmensity  1
490 159 mmensity o 1
490 160  ensity of 1
490 161  nsity of  1
490 162  dity of t 1
490 163 dity 

494 61 tollect wo 1
494 62 hllect woo 1
494 63  lect wood 1
494 64 eect wood  1
494 65 e t wood a 1
494 66  t wood an 1
494 67 h wood and 1
494 68 haood and  1
494 69 tood and d 1
494 70 ood and do 1
494 71  d and don 1
494 72   and don' 1
494 73  a d don't 1
494 74 tnd don't  1
494 75 nd won't a 1
494 76 d don't as 1
494 77  aon't ass 1
494 78 ton't dssi 1
494 79  n't dssig 1
494 80  't dssign 1
494 81 dt dssign  1
494 82 t dssign t 1
494 83 hassign th 1
494 84 tnsign the 1
494 85 nsign them 1
494 86 iign them  1
494 87 ign them t 1
494 88 gn them ta 1
494 89 n them tas 1
494 90 dthem task 1
494 91 toem tosks 1
494 92 her tosks  1
494 93 em tosks a 1
494 94   tosks an 1
494 95  tosks and 1
494 96 tosks and  1
494 97 hsks and w 1
494 98 nss and wo 1
494 99 is and wor 1
494 100 , and work 1
494 101 iand work, 1
494 102 tnd dork,  1
494 103 nd work, b 1
494 104 d dork, bu 1
494 105  aork, but 1
494 106 took, but  1
494 107 ook, but r 1
494 108  k, but ra 1
494 109  , but rat 1
494 110 , 

497 51  ether to  1
497 52 nther to c 1
497 53  her to co 1
497 54  er te col 1
497 55 em te coll 1
497 56   te colle 1
497 57  th collec 1
497 58 to lollect 1
497 59   lollect  1
497 60  lollect w 1
497 61 tollect wo 1
497 62 tllect woo 1
497 63  lect wood 1
497 64 eect wood  1
497 65 e t wood a 1
497 66  t wood an 1
497 67 t wood and 1
497 68  tood and  1
497 69 tood and d 1
497 70 ood and do 1
497 71  d and don 1
497 72   and don' 1
497 73  a d don't 1
497 74 tnd don't  1
497 75 nd don't a 1
497 76 d don't as 1
497 77  aon't ass 1
497 78 ton't dssi 1
497 79  n't dssig 1
497 80  't dssign 1
497 81 dt dssign  1
497 82 t dssign t 1
497 83  tssign th 1
497 84 tnsign the 1
497 85 nsign them 1
497 86  ign them  1
497 87  tn them t 1
497 88 fn them ta 1
497 89 n them tas 1
497 90 dthem task 1
497 91 toem tasks 1
497 92  er tasks  1
497 93 em tasks a 1
497 94   tasks an 1
497 95  tasks and 1
497 96 tosks and  1
497 97  sks and w 1
497 98 nss and wo 1
497 99  s and wor 1
497 100 , and work 1

In [12]:
results = sess.run(outputs, feed_dict={X: dataX})

for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

m you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.